In [ ]:
import re
import os
import numpy as np
import pandas as pd
import h5py
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import matplotlib.pyplot as plt
from tqdm import tqdm

# Function to convert string representations using Mousavi's method
def string_convertor(dd):
    dd2 = dd.split()
    SNR = []
    for i, d in enumerate(dd2):
        if d != '[' and d != ']':
            dL = d.split('[')
            dR = d.split(']')
            if len(dL) == 2:
                dig = dL[1]
            elif len(dR) == 2:
                dig = dR[0]
            elif len(dR) == 1 and len(dR) == 1:
                dig = d
            try:
                dig = float(dig)
            except Exception:
                dig = None
            SNR.append(dig)
    return SNR

# Dataset class
class EarthquakeDataset(Dataset):
    def __init__(self, file_name, file_list):
        self.file_name = file_name
        self.file_list = file_list
        self.X = []
        self.Y = []
        self.metadata = {
            'net_code': [], 'rec_type': [], 'eq_id': [], 'eq_depth': [], 'eq_mag': [],
            'mag_type': [], 'mag_auth': [], 'eq_dist': [], 'snr': [], 'trace_name': [],
            'S_P': [], 'baz': []
        }
        self.load_data()

    def load_data(self):
        with h5py.File(self.file_name, 'r') as dtfl:
            for evi in tqdm(self.file_list):
                dataset = dtfl.get(f'earthquake/local/{evi}')
                if dataset is None:
                    print(f"Dataset not found for event ID: {evi}")
                    continue
                data = np.array(dataset)
                spt = int(dataset.attrs['p_arrival_sample'])
                if spt < 100 or spt + 2900 > len(data):
                    print(f"Invalid sample range for event ID: {evi}")
                    continue
                dshort = data[spt-100:spt+2900, :]
                self.X.append(dshort)
                mag = round(float(dataset.attrs['source_magnitude']), 2)
                self.Y.append(mag)

                # Extract additional metadata
                self.metadata['net_code'].append(dataset.attrs['network_code'])
                self.metadata['rec_type'].append(dataset.attrs['receiver_type'])
                self.metadata['eq_id'].append(dataset.attrs['source_id'])
                self.metadata['eq_depth'].append(dataset.attrs['source_depth_km'])
                self.metadata['eq_mag'].append(mag)
                self.metadata['mag_type'].append(dataset.attrs['source_magnitude_type'])
                self.metadata['mag_auth'].append(dataset.attrs['source_magnitude_author'])
                self.metadata['eq_dist'].append(round(float(dataset.attrs['source_distance_deg']), 2))
                self.metadata['snr'].append(round(np.mean(dataset.attrs['snr_db']), 2))
                self.metadata['trace_name'].append(dataset.attrs['trace_name'])
                self.metadata['S_P'].append(int(dataset.attrs['s_arrival_sample']) - spt)
                self.metadata['baz'].append(round(float(dataset.attrs['back_azimuth_deg']), 2))

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return (
            torch.tensor(self.X[idx], dtype=torch.float32),
            torch.tensor(self.Y[idx], dtype=torch.float32),
            self.metadata['net_code'][idx],
            self.metadata['rec_type'][idx],
            self.metadata['eq_id'][idx],
            self.metadata['eq_depth'][idx],
            self.metadata['eq_mag'][idx],
            self.metadata['mag_type'][idx],
            self.metadata['mag_auth'][idx],
            self.metadata['eq_dist'][idx],
            self.metadata['snr'][idx],
            self.metadata['trace_name'][idx],
            self.metadata['S_P'][idx],
            self.metadata['baz'][idx]
        )


# Define the paths to your data files
file_name = "/content/drive/My Drive/2023-2024/UCL MSc in DSML/Term 3/MSc Project/Code/Datasets/STEAD/merge.hdf5"
csv_file = "/content/drive/My Drive/2023-2024/UCL MSc in DSML/Term 3/MSc Project/Code/Datasets/STEAD/merge.csv"

# Verify file existence
assert os.path.isfile(file_name), f"HDF5 file not found at {file_name}"
assert os.path.isfile(csv_file), f"CSV file not found at {csv_file}"

# Load and filter the dataset
df = pd.read_csv(csv_file, low_memory=False)
print(f"Initial number of records: {len(df)}")

# Filtering the dataset
df = df[df.trace_category == 'earthquake_local']
df = df[df.source_distance_km <= 110]
df = df[df.source_magnitude_type == 'ml']
df = df[df.p_arrival_sample >= 200]
df = df[df.p_arrival_sample + 2900 <= 6000]
df = df[df.p_arrival_sample <= 1500]
df = df[df.s_arrival_sample >= 200]
df = df[df.s_arrival_sample <= 2500]

# Fix coda_end_sample column parsing
df['coda_end_sample'] = df['coda_end_sample'].apply(lambda x: float(x.strip('[]')))
df = df.dropna(subset=['coda_end_sample'])
df = df[df['coda_end_sample'] <= 3000]

df = df[df.p_travel_sec.notnull()]
df = df[df.p_travel_sec > 0]
df = df[df.source_distance_km.notnull()]
df = df[df.source_distance_km > 0]
df = df[df.source_depth_km.notnull()]
df = df[df.source_magnitude.notnull()]
df = df[df.back_azimuth_deg.notnull()]
df = df[df.back_azimuth_deg > 0]
df['snr_db'] = df['snr_db'].apply(lambda x: np.mean(string_convertor(x)))
df = df[df.snr_db >= 20]

# Implementing multi-observations with a threshold of >= 400
uniq_ins = df.receiver_code.unique()

labM = []
for ii in range(0, len(uniq_ins)):
    print(str(uniq_ins[ii]), sum(n == str(uniq_ins[ii]) for n in df.receiver_code))
    stn = sum(n == str(uniq_ins[ii]) for n in df.receiver_code)
    if stn >= 400:
        labM.append(str(uniq_ins[ii]))

np.save('multi_observations', labM)

multi_observations = np.load('multi_observations.npy')
ev_list = []
for index, row in df.iterrows():
    st = row['receiver_code']
    if st in multi_observations:
        ev_list.append(row['trace_name'])

ev_list = df.trace_name.tolist()
np.random.shuffle(ev_list)  

# Splitting data into training, validation, and test sets
training = ev_list[:int(0.7*len(ev_list))]
validation = ev_list[int(0.7*len(ev_list)): int(0.8*len(ev_list))]
test = ev_list[int(0.8*len(ev_list)):]

# Creating datasets and data loaders
train_dataset = EarthquakeDataset(file_name, training)
val_dataset = EarthquakeDataset(file_name, validation)
test_dataset = EarthquakeDataset(file_name, test)

batch_size = 256

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Define the PyTorch model
class EarthquakeModel(nn.Module):
    def __init__(self, dropout_rate):
        super(EarthquakeModel, self).__init__()
        self.conv1 = nn.Conv1d(3, 64, kernel_size=3, padding='same')
        self.dropout1 = nn.Dropout(dropout_rate)
        self.pool1 = nn.MaxPool1d(4, padding=1)
        
        self.conv2 = nn.Conv1d(64, 32, kernel_size=3, padding='same')
        self.dropout2 = nn.Dropout(dropout_rate)
        self.pool2 = nn.MaxPool1d(4, padding=1)

        self.lstm = nn.LSTM(32, 100, bidirectional=True, batch_first=True)

        self.fc = nn.Linear(200, 2)  # 2 for mean and variance
    
    def forward(self, x):
        x = x.permute(0, 2, 1)  # Change the shape from (batch, seq, features) to (batch, features, seq)
        x = self.conv1(x)
        x = self.dropout1(x)
        x = self.pool1(x)
        
        x = self.conv2(x)
        x = self.dropout2(x)
        x = self.pool2(x)

        x = x.permute(0, 2, 1)  # Change the shape back to (batch, seq, features) for LSTM
        x, _ = self.lstm(x)

        x = self.fc(x[:, -1, :])  # Use the output of the last time step
        return x

# Custom loss function for aleatoric uncertainty
def custom_loss(y_true, y_pred):
    y_hat = y_pred[:, 0]
    s = y_pred[:, 1]
    return torch.mean(0.5 * torch.exp(-s) * (y_true - y_hat) ** 2 + 0.5 * s)

# Create the model
drop_rate = 0.2
model = EarthquakeModel(dropout_rate=drop_rate)

# Define optimizer and learning rate scheduler
optimizer = optim.Adam(model.parameters(), lr=0.001)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=0.1, patience=4, min_lr=0.5e-6)

# Training function
def train(model, train_loader, optimizer, epoch):
    model.train()
    train_loss = 0
    for batch_idx, (data, target, *_) in enumerate(train_loader):  # ignore metadata
        optimizer.zero_grad()
        output = model(data)
        loss = custom_loss(target, output)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
    train_loss /= len(train_loader.dataset)
    print(f'Train Epoch: {epoch} \tLoss: {train_loss:.6f}')
    return train_loss

# Validation function
def validate(model, val_loader):
    model.eval()
    val_loss = 0
    with torch.no_grad():
        for data, target, *_) in val_loader:  # ignore metadata
            output = model(data)
            val_loss += custom_loss(target, output).item()
    val_loss /= len(val_loader.dataset)
    print(f'Validation set: Average loss: {val_loss:.6f}\n')
    return val_loss

# Training and validation loop
epochs = 200
train_losses = []
val_losses = []
best_val_loss = float('inf')

for epoch in range(1, epochs + 1):
    train_loss = train(model, train_loader, optimizer, epoch)
    val_loss = validate(model, val_loader)
    scheduler.step(val_loss)
    
    train_losses.append(train_loss)
    val_losses.append(val_loss)
    
    # Save model if validation loss decreases
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save(model.state_dict(), 'best_model.pt')

# Define the MCDropoutModel for uncertainty estimation
class MCDropoutModel(nn.Module):
    def __init__(self, model, n_iter=50):
        super(MCDropoutModel, self).__init__()
        self.model = model
        self.n_iter = n_iter

    def forward(self, x):
        self.model.train()
        preds = []
        for _ in range(self.n_iter):
            preds.append(self.model(x))
        preds = torch.stack(preds)
        return preds

# Function to compute uncertainties
def compute_uncertainties(preds):
    pred_mean = preds[:, :, 0].mean(dim=0)
    pred_var = preds[:, :, 0].var(dim=0)
    aleatoric_unc = preds[:, :, 1].mean(dim=0)
    epistemic_unc = pred_var
    combined_unc = aleatoric_unc + epistemic_unc
    return pred_mean, aleatoric_unc, epistemic_unc, combined_unc

# Load the best model
model.load_state_dict(torch.load('best_model.pt'))

# Create MCDropoutModel
mc_model = MCDropoutModel(model)

# Test the model and compute uncertainties
test_preds = []
test_targets = []
test_metadata = {key: [] for key in train_dataset.metadata.keys()}

with torch.no_grad():
    for data, target, *metadata in test_loader:
        preds = mc_model(data)
        test_preds.append(preds)
        test_targets.append(target)
        
        for key, value in zip(test_metadata.keys(), metadata):
            test_metadata[key].append(value)

test_preds = torch.cat(test_preds, dim=1)
test_targets = torch.cat(test_targets)

pred_mean, aleatoric_unc, epistemic_unc, combined_unc = compute_uncertainties(test_preds)

# Plotting and saving results
# Plotting training and validation loss
plt.figure()
plt.plot(train_losses, label='Train Loss')
plt.plot(val_losses, label='Validation Loss')
plt.legend()
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training and Validation Loss')
plt.grid()
plt.savefig('loss_curve.png')

# Plotting uncertainties
fig1 = plt.figure()
plt.errorbar(pred_mean, aleatoric_unc, xerr=aleatoric_unc, fmt='o', alpha=0.4, ecolor='g', capthick=2)
plt.plot(test_targets, aleatoric_unc, 'ro', alpha=0.4)
plt.xlabel('Magnitude')
plt.ylabel('Aleatoric Uncertainty')
plt.title('Aleatoric Uncertainty')
fig1.savefig('aleatoric_uncertainty.png')

fig2 = plt.figure()
plt.errorbar(pred_mean, epistemic_unc, xerr=epistemic_unc, fmt='o', alpha=0.4, ecolor='g', capthick=2)
plt.plot(test_targets, epistemic_unc, 'ro', alpha=0.4)
plt.xlabel('Magnitude')
plt.ylabel('Epistemic Uncertainty')
plt.title('Epistemic Uncertainty')
fig2.savefig('epistemic_uncertainty.png')

fig3 = plt.figure()
plt.errorbar(pred_mean, combined_unc, xerr=combined_unc, fmt='o', alpha=0.4, ecolor='g', capthick=2)
plt.plot(test_targets, combined_unc, 'ro', alpha=0.4)
plt.xlabel('Magnitude')
plt.ylabel('Combined Uncertainty')
plt.title('Combined Uncertainty')
fig3.savefig('combined_uncertainty.png')

fig4, ax = plt.subplots()
ax.scatter(test_targets, pred_mean, alpha=0.4, facecolors='none', edgecolors='r')
ax.plot([test_targets.min(), test_targets.max()], [test_targets.min(), test_targets.max()], 'k--', alpha=0.4, lw=2)
ax.set_xlabel('Measured')
ax.set_ylabel('Predicted')
fig4.savefig('measured_vs_predicted.png')

# Save additional metadata
for key in test_metadata.keys():
    test_metadata[key] = [item for sublist in test_metadata[key] for item in sublist]

# Saving results to CSV
results_df = pd.DataFrame({
    'true_magnitude': test_targets.numpy(),
    'predicted_magnitude': pred_mean.numpy(),
    'aleatoric_uncertainty': aleatoric_unc.numpy(),
    'epistemic_uncertainty': epistemic_unc.numpy(),
    'combined_uncertainty': combined_unc.numpy(),
    'net_code': test_metadata['net_code'],
    'rec_type': test_metadata['rec_type'],
    'eq_id': test_metadata['eq_id'],
    'eq_depth': test_metadata['eq_depth'],
    'eq_mag': test_metadata['eq_mag'],
    'mag_type': test_metadata['mag_type'],
    'mag_auth': test_metadata['mag_auth'],
    'eq_dist': test_metadata['eq_dist'],
    'snr': test_metadata['snr'],
    'trace_name': test_metadata['trace_name'],
    'S_P': test_metadata['S_P'],
    'baz': test_metadata['baz']
})
results_df.to_csv('results.csv', index=False)

print('Results have been saved successfully.')
